# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [2]:
# usar o: Chrome -> chromedriver
# firefox -> geckodriver

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()

# Passo 1:Pegar a cotação do dolar
# Entrar no google
navegador.get("https://www.google.com/")

# Para usar o xpath eu ultilizo 3 métodos: .click() -> clicar , .get() -> pegar uma informação, .send_keys() -> para escrever 

navegador.find_element('xpath', 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar") 

navegador.find_element(
    'xpath',
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)


cotacao_dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

# Passo 2:Pegar a cotação do euro

navegador.get("https://www.google.com/")

# Para usar o xpath eu ultilizo 3 métodos: .click() -> clicar , .get() -> pegar uma informação, .send_keys() -> para escrever

navegador.find_element('xpath', 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro") 

navegador.find_element(
    'xpath',
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)


cotacao_euro = navegador.find_element('xpath',
                                      '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

# Passo 3:Pegar a cotação do ouro

navegador.get("https://www.melhorcambio.com/ouro-hoje#:~:text=O%20valor%20do%20grama%20do,%C3%A9%20de%20car%C3%A1ter%20exclusivamente%20informativo.")

cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()#pra fechar a aba


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input"}
  (Session info: chrome=111.0.5563.147)
Stacktrace:
Backtrace:
	(No symbol) [0x0061DCE3]
	(No symbol) [0x005B39D1]
	(No symbol) [0x004C4DA8]
	(No symbol) [0x004F019F]
	(No symbol) [0x004F03AB]
	(No symbol) [0x0051EE62]
	(No symbol) [0x0050AF14]
	(No symbol) [0x0051D57C]
	(No symbol) [0x0050ACC6]
	(No symbol) [0x004E6F68]
	(No symbol) [0x004E80CD]
	GetHandleVerifier [0x00893832+2506274]
	GetHandleVerifier [0x008C9794+2727300]
	GetHandleVerifier [0x008CE36C+2746716]
	GetHandleVerifier [0x006C6690+617600]
	(No symbol) [0x005BC712]
	(No symbol) [0x005C1FF8]
	(No symbol) [0x005C20DB]
	(No symbol) [0x005CC63B]
	BaseThreadInitThunk [0x76347D69+25]
	RtlInitializeExceptionChain [0x7746B74B+107]
	RtlClearBits [0x7746B6CF+191]


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [ ]:
# Passo 4: Importar a base de dados e Atualizar a base
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [ ]:
# Passo 5: Recalcular os preços 

# atualizar a cotação
# cotação dolar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# preco de compra = cotação * preco original 
tabela["Preço de Compra"] * tabela["Preço Original"]

# preco de venda == preco de compra * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.087700,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.511785,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.087700,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.087700,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.511785,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.087700,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,315.120000,7000.00,1.15,8050.000


### Agora vamos exportar a nova base de preços atualizada

In [ ]:
# Passo 6: Exportar a base atualizada
tabela.to_excel("Produtos Novo.xlsx", index=False)

In [ ]:
# tem que baixar
#!pip install selenium